# TODO 
- [ ] Add illustrations for the concept (dinov2, instance retrieval, faiss)
- [ ] There are a lot of possible optimizations I can apply for both search and encoding, but I don't have time for now
- [ ] Add a pipeline to give a nice caption according to the input image and the nearest image with NLP 
- [ ] Upload it into HuggingFace space as a demo 
- [ ] Save Faiss state

# References & useful resources 


In [5]:
# !git clone https://github.com/facebookresearch/dinov2.git

In [6]:
# %cd /dinov2
# !pip install -r requirements.txt
# !pip install faiss-gpu

In [7]:
# !pip install faiss-gpu
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import faiss
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
import warnings

warnings.filterwarnings("ignore")
import gradio as gr

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the DINO model
dino = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")
dino.to(device)
dino.eval()

# Define the image transformations
image_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# Function to extract features from an image
def extract_features(image):
    image_tensor = image_transforms(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = dino(image_tensor).float()
    return features


# Function to perform nearest neighbor search
def nearest_neighbor_search(query_features, dataset_features, k=5):
    index = faiss.IndexFlatL2(query_features.shape[1])
    dataset_features = torch.cat(dataset_features, dim=0).cpu().numpy()
    index.add(dataset_features)
    distances, indices = index.search(query_features.cpu().numpy(), k)
    return indices


# Function to return the nearest neighbor images
def return_nearest_neighbor_images(query_image):
    query_features = extract_features(query_image)
    indices = nearest_neighbor_search(query_features, dataset_features)
    nearest_neighbor_image_paths = []
    for i in range(k):
        image_path = os.path.join(dataset_path, dataset_images[indices[0][i]])
        nearest_neighbor_image_paths.append(image_path)
    return nearest_neighbor_image_paths


# Gradio interface
dataset_path = "/home/kareem/Desktop/dinv2_Image_clustering/level_up_with_dinov2/few-shot-art-painting/data"
dataset_images = os.listdir(dataset_path)
dataset_features = []
for filename in dataset_images:
    image_path = os.path.join(dataset_path, filename)
    if (
        filename.endswith(".png")
        or filename.endswith(".jpg")
        or filename.endswith(".jpeg")
    ):
        image = Image.open(image_path)
        features = extract_features(image)
        dataset_features.append(features)

k = 5  # Number of nearest neighbors to retrieve

inputs = gr.inputs.Image(type="pil", label="Input Image")
# outputs = gr.outputs.Image(type="pil", label="Returned Images", multiple=True)

outputs = gr.Gallery()

demo = gr.Interface(
    fn=return_nearest_neighbor_images,
    inputs=inputs,
    outputs=outputs,
    title="Nearest Neighbor Images",
    description="Upload an image and get the nearest neighbor images from the dataset as output.",
)

demo.launch()

Using cache found in /home/kareem/.cache/torch/hub/facebookresearch_dinov2_main


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/kareem/mambaforge/envs/dinov2/lib/python3.9/site-packages/gradio/routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
  File "/home/kareem/mambaforge/envs/dinov2/lib/python3.9/site-packages/gradio/blocks.py", line 1384, in process_api
    result = await self.call_function(
  File "/home/kareem/mambaforge/envs/dinov2/lib/python3.9/site-packages/gradio/blocks.py", line 1089, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/kareem/mambaforge/envs/dinov2/lib/python3.9/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/kareem/mambaforge/envs/dinov2/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/home/kareem/mambaforge/envs/dinov2/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 807, in run
    result = contex